In [1]:
import libs.async_native as asn
import requests as req
import pymongo as pdb
import re
from datetime import datetime
from itertools import chain



In [ ]:
import json

In [2]:
def get_proxyscrape_proxies():
    def get_proxy_list (url,protocol=""):
        resp = req.get(url)
        return [protocol+i.strip() for i in resp.text.splitlines()]
    http   = get_proxy_list('https://api.proxyscrape.com/?request=getproxies&proxytype=http&timeout=10000&country=all&ssl=all&anonymity=all',
                            'http://')
    socks4 = get_proxy_list('https://api.proxyscrape.com/?request=getproxies&proxytype=socks4&timeout=10000&country=all',
                            'socks4://')
    socks5 = get_proxy_list('https://api.proxyscrape.com/?request=getproxies&proxytype=socks5&timeout=10000&country=all',
                            'socks5://')
    return http,socks4,socks5

In [39]:
ip_format = re.compile(r'(\d{,3}\.?){4}\s*')

def time_proxy(proxy, timeout=13, show_len=60, debug=False):
    ses = req.session()
    ses.proxies = {'http':proxy}
    
    try:
        resp = ses.get('http://checkip.amazonaws.com',timeout=timeout)

        
        #Security measures.
        if len(resp.text) > resp_len:
            _ = print(f"expected string < {resp_len} chars from {proxy}, got {len(resp.text)} chars.") \
            if debug else None
            return -15
        
        if not re.fullmatch(ip_format,resp.text):
            _ = print(f"proxy {proxy} sent back {resp.text.strip()} which was not a IP") \
            if debug else None
            return -16
        
        # handling code
        elapsed=resp.elapsed.total_seconds()
        _ =  print(f"{proxy} responded {resp.text.strip()} after {elapsed} seconds") \
        if debug else None
        
        
        
    except req.exceptions.Timeout:
        elapsed= -1
    except req.exceptions.ProxyError:
        elapsed= -2
    except ConnectionError:
        elapsed= -3
    except req.exceptions.ConnectionError:
        elapsed= -4
    except req.exceptions.ChunkedEncodingError:
        elapsed= -5
    except Exception as e:
        _ = print(f"Unhandled exception type {type(e)} raised by {proxy}")\
        if debug else None
        elapsed= -10
    return (elapsed)



def tp_job (iter_proxy,timeout=13):
    
    return [lambda p=proxy, t=timeout:(time_proxy(p,t),p) for proxy in iter_proxy]

def tp_multi (iter_proxy, attempts=10, timeout=13):
    time_reccord = [(i,list()) for i in iter_proxy]
    job_list = [lambda p=proxy, t=timeout, x=target:x.append(time_proxy(p,t)) for proxy,target in time_reccord]
    return job_list * attempts, time_reccord

sort_prox = lambda out: sorted([i for i in out[0] if i[0] > 0], key= lambda j: j[0] )
multi_prox_sort = lambda times: sorted([i for i in times if min(i[1]) > 0], key=lambda j: sum(j[1]))

def multi_prox_sort (results):
    
    for proxy,times in results:
        if min(times) < 0:
            continue
        
        

In [18]:
p = list(chain(*get_proxyscrape_proxies()))

In [27]:
jobs, times = tp_multi(p,10)

In [28]:
times

[('http://1.20.99.89:32963', []),
 ('http://005.2.72.101:3128', []),
 ('http://1.10.188.42:48721', []),
 ('http://1.10.189.84:35138', []),
 ('http://1.20.101.149:44778', []),
 ('http://01.10.188.202:8080', []),
 ('http://101.255.125.124:8080', []),
 ('http://1.10.188.132:8080', []),
 ('http://114.103.21.94:4216', []),
 ('http://114.103.138.178:1080', []),
 ('http://116.17.102.74:3128', []),
 ('http://115.219.134.44:3000', []),
 ('http://114.99.15.143:3000', []),
 ('http://114.103.20.36:4216', []),
 ('http://115.219.134.143:3000', []),
 ('http://116.203.22.243:80', []),
 ('http://13.251.221.173:8080', []),
 ('http://128.199.202.122:8080', []),
 ('http://128.199.202.122:3128', []),
 ('http://14.207.149.81:8080', []),
 ('http://139.228.32.212:8080', []),
 ('http://128.14.178.94:3128', []),
 ('http://136.243.81.120:80', []),
 ('http://13.209.83.171:3128', []),
 ('http://184.82.228.15:2020', []),
 ('http://183.250.255.86:63000', []),
 ('http://183.164.227.180:8888', []),
 ('http://183.166.1

In [ ]:
out = asn.concurrent_exec(jobs,workers=300)

In [45]:
reliable = [i for i in times if min(i[1]) > 0]

In [59]:
rel = sorted(reliable, key= lambda j: sum(j[1]))

In [60]:
print(rel)

[('socks4://178.249.219.91:4145', [0.613876, 0.64776, 0.625446, 0.699927, 0.65888, 0.623992, 0.638236, 0.602073, 0.658746, 0.596618]), ('socks4://213.251.255.170:61586', [0.715624, 0.698319, 1.742243, 0.708105, 0.741212, 0.717186, 0.695586, 0.768624, 0.703842, 0.696321]), ('socks4://174.75.211.222:4145', [0.557021, 0.538259, 0.536912, 0.600321, 0.690268, 3.645556, 0.551367, 0.563504, 0.544763, 0.526704]), ('socks4://200.41.182.243:4145', [0.876824, 0.85529, 0.867229, 0.970048, 0.915454, 0.878696, 0.885955, 0.873067, 0.885867, 0.877365]), ('socks4://178.48.68.61:4145', [0.98196, 0.960616, 0.985574, 1.013978, 1.005098, 0.96473, 0.984021, 1.017679, 0.970095, 0.958295]), ('socks4://51.178.81.43:1080', [0.541333, 0.54643, 1.014005, 1.248475, 1.447467, 0.997133, 0.969508, 1.141411, 1.50239, 0.583218]), ('socks4://179.60.127.45:59366', [0.793737, 0.798208, 0.847174, 1.794107, 0.796481, 1.803966, 0.953992, 1.791368, 1.760671, 0.737407]), ('socks5://198.13.63.193:1081', [1.178956, 1.106719, 1.0

In [7]:
good_prox = sort_prox(out)

TypeError: '>' not supported between instances of 'tuple' and 'int'

In [10]:
good_prox

[(0.104924, 'socks4://159.89.237.201:40001'),
 (0.161809, 'http://159.203.61.169:3128'),
 (0.182889, 'http://159.203.61.169:8080'),
 (0.211518, 'socks5://198.46.205.105:1080'),
 (0.22402, 'socks4://192.158.15.201:50877'),
 (0.229326, 'http://45.62.216.124:3128'),
 (0.30691, 'socks4://50.27.184.225:54321'),
 (0.318756, 'socks4://50.197.210.138:32100'),
 (0.334704, 'http://165.22.36.75:8888'),
 (0.335182, 'http://102.129.249.120:3128'),
 (0.342848, 'socks4://64.207.95.113:39593'),
 (0.364538, 'http://102.129.249.120:8080'),
 (0.388513, 'socks4://181.189.235.134:50488'),
 (0.40375, 'socks4://24.139.143.226:4153'),
 (0.417081, 'socks4://191.103.219.225:44673'),
 (0.419237, 'socks4://181.129.70.82:44357'),
 (0.439776, 'http://161.97.72.126:3128'),
 (0.442165, 'socks4://181.129.5.34:4153'),
 (0.449698, 'socks5://173.254.222.170:1085'),
 (0.457982, 'socks4://181.129.62.2:47377'),
 (0.458632, 'socks4://200.52.144.170:33865'),
 (0.466311, 'http://161.35.70.249:3128'),
 (0.473769, 'socks4://190.